In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable 
import pandas as pd
from scipy import signal
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import random
#import seaborn as sns
"""
    number of trajectories in each city
    # austin --  train: 43041 test: 6325 
    # miami -- train: 55029 test:7971
    # pittsburgh -- train: 43544 test: 6361
    # dearborn -- train: 24465 test: 3671
    # washington-dc -- train: 25744 test: 3829
    # palo-alto -- train:  11993 test:1686

    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds
    
"""

'\n    number of trajectories in each city\n    # austin --  train: 43041 test: 6325 \n    # miami -- train: 55029 test:7971\n    # pittsburgh -- train: 43544 test: 6361\n    # dearborn -- train: 24465 test: 3671\n    # washington-dc -- train: 25744 test: 3829\n    # palo-alto -- train:  11993 test:1686\n\n    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds\n    \n'

In [2]:
# clean memory
import gc

gc.collect()

torch.cuda.empty_cache()

In [3]:
print(torch.cuda.get_device_name())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GeForce RTX 2060 SUPER


In [4]:
from glob import glob
import pickle
import numpy as np

### Change to requried path to access data locally, too big too push all data into github
#ROOT_PATH = "C:/Users/Administrator/cse151b-spring2022/argo2/"
ROOT_PATH = "D:/School/cse151B/argo2/"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False):
    f_in = ROOT_PATH + split + "/" + city + "_inputs"
    inputs = pickle.load(open(f_in, "rb"))
    inputs = np.asarray(inputs)
    
    outputs = None
    
    if split=="train":
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)

        return torch.from_numpy(inputs).float(), torch.from_numpy(outputs).long()

    if split=="test":
    
        return torch.from_numpy(inputs).float(), torch.from_numpy(np.array([]))

    

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str, transform=None, device='cpu'):
        super(ArgoverseDataset, self).__init__()
        self.transform = transform
        self.split = split
        self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=False)
        self.inputs = self.inputs.to(device)
        self.outputs = self.outputs.to(device)
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):

        if self.split == 'train':
            data = (self.inputs[idx], self.outputs[idx])
        if self.split == 'test':
            data = self.inputs[idx]
            
        if self.transform:
            data = self.transform(data)

        return data

# intialize a dataset
city = 'palo-alto' 
split = 'train'
#train_dataset  = ArgoverseDataset(city = city, split = split, device=device)

In [5]:
#len(train_dataset) # current X is 50 in len and Y is 60 in len

11993

In [6]:
#type(train_dataset)

__main__.ArgoverseDataset

### Data Preprocessing

In [7]:
# wrapper function to select proportion of a cities random examples w/o replacement from each city and put all data into one list
# purpose: whole dataset is too big and might be redundant
def randomCitySampler(prop):
    cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc","palo-alto"]

    samples = []
    for c in cities:
        # get city data
        temp_dataset = ArgoverseDataset(city = c, split = "train", device=device)

        numProp = int(len(temp_dataset) * prop)

        # get N number of random indicies
        ind = random.sample(range(0, len(temp_dataset)), numProp)
        #print(ind)
        # push all data indicies into samples list
        for i in ind: 
            samples.append(temp_dataset[i])
    return samples


In [8]:
### constants for generating Dataset
proportionOfEntireData = 0.8
seqLen = 40
stepSize = 3
batch_sz = 128  # batch size 

In [9]:
# create train dataset, with proportion to actual amount data
sampleTest = randomCitySampler(proportionOfEntireData)
len(sampleTest)

40760

In [10]:
# generate sequences of length seqLength and specific step size
def sequenceGenerator(data, seqLen=40, stepSize=5):
    newData = []
    for d in data:
        # concat X and Y together
        temp = torch.cat([d[0],d[1]])
        # make X of length SeqLen and Y is next x,y coordinate pair
        for i in range(0,len(temp)-seqLen, stepSize):
            x = temp[i:i + seqLen]
            #flatX = Variable(torch.tensor([item for sublist in x for item in sublist])).to(device)
            flatX = torch.flatten(Variable(torch.tensor(x)).to(device))
            y = temp[i+seqLen]
            newData.append((flatX,y))
            
        
    return newData

In [11]:
# generate sequences
train_seq_data = sequenceGenerator(sampleTest,seqLen, stepSize)
len(train_seq_data)

<ipython-input-10-8e56f7b34fb3>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  flatX = torch.flatten(Variable(torch.tensor(x)).to(device))


570640

In [12]:
# should be a vector of size (80,2) for each example
train_seq_data[0], len(train_seq_data[0][0])

((tensor([-190.3386, -646.6285, -190.3374, -646.6282, -190.3361, -646.6277,
          -190.3349, -646.6282, -190.3335, -646.6281, -190.3322, -646.6283,
          -190.3309, -646.6285, -190.3298, -646.6289, -190.3288, -646.6285,
          -190.3286, -646.6292, -190.3291, -646.6304, -190.3303, -646.6332,
          -190.3321, -646.6367, -190.3340, -646.6390, -190.3366, -646.6422,
          -190.3391, -646.6442, -190.3418, -646.6453, -190.3443, -646.6448,
          -190.3469, -646.6437, -190.3495, -646.6417, -190.3526, -646.6406,
          -190.3557, -646.6402, -190.3584, -646.6381, -190.3614, -646.6367,
          -190.3641, -646.6353, -190.3668, -646.6343, -190.3695, -646.6334,
          -190.3728, -646.6352, -190.3758, -646.6373, -190.3786, -646.6401,
          -190.3815, -646.6432, -190.3838, -646.6456, -190.3864, -646.6494,
          -190.3883, -646.6514, -190.3897, -646.6524, -190.3908, -646.6529,
          -190.3917, -646.6522, -190.3926, -646.6522, -190.3932, -646.6509,
          -1

In [13]:
# create loader
train_loader = DataLoader(train_seq_data,batch_size=batch_sz)

In [14]:
# check shape is correct
train_features, train_labels = next(iter(train_loader))
len(train_features[0]), train_labels[0]
# shape is correct

(80, tensor([-190.3935, -646.6465], device='cuda:0'))

### LSTM

In [15]:
# model parameters
#num_epochs = 60
learning_rate = 0.001

input_size = seqLen*2 #number of features
hidden_size = 150 #number of features in hidden state
num_layers = 2 #number of stacked lstm layers

output_size = 2 #number of output classes 


In [16]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size,num_layers):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size,num_layers)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(num_layers,1,self.hidden_layer_size).to(device),
                            torch.zeros(num_layers,1,self.hidden_layer_size).to(device))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        
        return predictions

In [17]:
lstm = LSTM(input_size, hidden_size,output_size,num_layers)
lstm = lstm.to(device)
lstm

LSTM(
  (lstm): LSTM(80, 100)
  (linear): Linear(in_features=100, out_features=2, bias=True)
)

In [18]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

### Training

In [19]:
epochs = 10

for i in range(epochs):
    for seq, labels in train_loader:
        lstm.train()
        seq = seq.to(device)
        labels = labels.to(device)

        

        optimizer.zero_grad()
        lstm.hidden_cell = (torch.zeros(num_layers, 1, lstm.hidden_layer_size).to(device),
                        torch.zeros(num_layers, 1, lstm.hidden_layer_size).to(device))
        
        
        y_pred = lstm(seq)
        
        #print(seq.shape)
        #print(y_pred.shape, labels.shape)
        #break

        loss = loss_function(y_pred, labels)
        loss.backward()
        optimizer.step()

    if i%3 == 1:
        print(f'epoch: {i:3} loss: {loss.item():10.8f}')

print(f'epoch: {i:3} loss: {loss.item():10.10f}')

epoch:   1 loss: 154246.62500000
epoch:   4 loss: 137028.45312500
epoch:   7 loss: 319852.68750000
epoch:   9 loss: 369894.4375000000


### Predicting

In [28]:
# need to cycle for each test set and use LSTM to predict 
# add results for each city to dataframe 
# concatinate all dataframes

def validation(model):
    cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc","palo-alto"]

    # all the data frames
    allDF = []
    with torch.no_grad():
        for c in cities:
            
            test_dataset = ArgoverseDataset(city=c, split='test', device=device)
            test_loader = DataLoader(test_dataset,batch_size=128)

            cityPredictions = []
            for t in test_loader.dataset:
                model.eval()
                flat = torch.flatten(t)
                currentPred = []

                for i in range(60):
                    #print(flat)
                    #print(len(flat)-seqLen*2)

                    pred = torch.flatten(model(flat[len(flat)-seqLen*2:].view(1,seqLen*2)))

                    #print(flat[len(flat)-seqLen*2:].view(1,80))
                    #print(pred)

                    currentPred.append(pred)

                    #print(torch.flatten(pred).shape)
                    #print(flat.shape)
                    
                    flat = torch.cat((flat,pred),0)
                    #print(flat)

                #print(len(flat[100:]))
                cityPredictions.append(flat[100:].detach().to('cpu').numpy())
        
            df = pd.DataFrame(cityPredictions)
            df.columns = ['v' + str(i) for i in (range(120))]
            df['ID'] = [str(i) + '_' + c for i in (range(len(test_loader.dataset)))]
            allDF.append(df)
            
    return allDF


In [29]:
tempValDF = validation(lstm)


In [30]:
len(tempValDF)

6

In [33]:
lstmPredFinal = pd.concat(tempValDF)
lstmPredFinal.ID

0             0_austin
1             1_austin
2             2_austin
3             3_austin
4             4_austin
             ...      
1681    1681_palo-alto
1682    1682_palo-alto
1683    1683_palo-alto
1684    1684_palo-alto
1685    1685_palo-alto
Name: ID, Length: 29843, dtype: object

In [35]:
lstmPredFinal.to_csv("lstm2.csv", index=False)